# Colin Lefter

## Research question/interests

**What equity data is the most deterministic of the price of an equity, and of this data, which is the most relevant for a growth portfolio investment strategy such that we can compute an optimized portfolio of equities while using user input to drive our optimization algorithm?**

My research objective is to develop a scalable asset allocation and construction algorithm that implements an objected-oriented design approach. This objective is an outcome of determining what equity data is the most deterministic of the price of an equity, which will be the focus for the majority of the project/

I intend to develop algorithms for constructing multiple linear regressions and Fourier Franforms, among others, that I will then use to construct interactive and statistical models with Plotly and Seaborn. As such, I have a strong interest in the system design of our software and in developing helper functions that can assist all of us with processing data more efficiently. I am also looking forward to using Facebook Prophet[^1] to construct a time series forecast of a sample portfolio recommendation from our software, which can be included in our Tableau Dashboard.

### Analysis Plan
Our objective function is one that takes in a selection of columns from our data sets to then search for the top n companies that satisfy a criteria for having the highest probability of producing an optimal return on investment. These inputs themselves refer to sub-objective functions that take as input user-defined parameters and thresholds that set the criteria for favourable performance attributes. To rank the companies from our data set, and ultimately determine what portion of capital to assign to each equity, I propose a data normalization algorithm that normalizes the data that comprises the favourable subset from each column of our data set. We interpret these normalized values as probabilities of equity selection and ultimately average the score of each company across all columns to then multiply the final score percentage of each company with the total capital specified by the user. In a broad sense, our software is composed of four general classes that include "Data", "Quantitative Analysis", "Data Visualization" and "Portfolio Construction". We inherit the properties from each of these classes to build a functional data analysis chain.

Our data visualization will be concerned with analyzing the influence of certain financial variables, such as Price-to-Earnings, on the price of each equity from a sample of 500 equities (from the S&P 500 index). Such analysis would begin with a statistical summary that will constitute exploratory data analysis, followed by our application of analysis algorithms that we design. The construction of a portfolio is a bonus of our project and will be made possible by the analysis algorithms we have constructed.

**Important Note**
A component of the analysis will involve the comparison of different values of financial variables with the corresponding price of each equity. This constitutes inferential analysis as we are attempting to identify a correlation on the basis of picking stocks based on expected performance. Therefore, this will require us to use past financial data and compare this data with the current price of each equity. As a result, we can only use the 3-month performance data (i.e. 3-month change in share price data) for this comparison as otherwise we would be using future data to predict past performance, which would be invalid.

#### User-defined parameters
Some initial ideas for these parameters include:
- (float) Initial capital
- (float) Additional capital per day, week or month
- (int) Intended holding period (in days)
- (boolean) Importance of dividends (validated based on capital invested)
- (String) Preferred industries (choose from a list, or select all)
- (int) Volatility tolerance (from 0 to 1, 1 indicating that volatility is not important)
- (String) Preferred companies (as a list)[^2]
- (int) Preferred degree of portfolio diversification (from 0 to 1, 1 indicating complete diversification)
- (String) Preferred investment strategy (choose from "Growth", "Value", "GARP")

### Algorithm Plan

####  Tier 1: Threshold-based screening algorithms
- The current plan is to use these algorithms to screen the financial documents from each company by setting a minimum threshold for each financial ratio. This class of algorithms will need to conduct such screening per industry as industry financial ratios are dinstinct from one another.
- A global screening algorithm that selects companies which show favourable performance across all ratios can also be used after each ratio has been individually tested.

#### Tier 2: Regression models
- As of now, the intent is to develop a multiple linear regression model that will attempt to determine a relationship between the yearly and quarterly performance of each company in relation to several columns of data that act as predictors. This can essentially implement the results from the threshold-based screening algorithms to only conduct this analysis on the pre-screened companies.

#### Tier 3: Statistical modelling algorithms
- Tier 3 denotes a class of broadly experimental statistical modelling algorithms that are applied on a pre-final portfolio to add additional points to companies that perform exceptionally well compared to others in the portfolio. For now, these algorithms constitute signal processing algorithms such as a Fourier Transform algorithm that attempts to identify peaks in numerical values that would otherwise not be apparent when examined in isolation and without further processing. Therefore, these algorithms will be used to fine-tune the capital allocation percentages for each company in the pre-final portfolio.

#### Columns of relevance
Data set 1: Overview
- Price
- MKT Cap
- P/E
- EPS
- Sector

Data set 2: Performance
- 1M change (1 month change)
- 3-Month performance
- 6-month perfromance
- YTD performance
- Yearly performance
- Volatility

Data set 3: Valuation
- Price / revenue
- Enterprise value

Data set 4: Dividends
- Dividend yield FWD
- Dividends per share (FY)

Data set 5: Margins
- Gross profit margin
- Operating margin
- Net profit margin

Data set 6: Income Statement
- Gross profit
- Income
- Net cash flow

Data set 7: Balance Sheet
- Current ratio
- Debt/equity
- Quick ratio

The total number of columns would be 24 in this case.

[^1]: This would mean that a few time series data sets would need to be downloaded from TradingView at the end of the project to test the demo porfolio.

[^2]: A helper function can be developed for this, where the user can just type out the name of the company and the ticker is identifed.

In [ ]:
import pandas as pd
import plotly as plt
import seaborn as sns
import numpy as np
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from IPython.display import display, HTML, Markdown, Latex
from tqdm import tqdm, trange
from typing import *
from dataclasses import dataclass
from scipy import stats

In [ ]:
# these variables will be updated to reflect the processed data at a later date
balance_sheet_df = pd.read_csv("../data/raw/us_equities_tradingview_data_balance_sheet.csv")
dividends_df = pd.read_csv("../data/raw/us_equities_tradingview_data_dividends.csv")
income_statement_df = pd.read_csv("../data/raw/us_equities_tradingview_data_income_statement.csv")
margins_df = pd.read_csv("../data/raw/us_equities_tradingview_data_margins.csv")
overview_df = pd.read_csv("../data/raw/us_equities_tradingview_data_overview.csv")
performance_df = pd.read_csv("../data/raw/us_equities_tradingview_data_performance.csv")
valuation_df = pd.read_csv("../data/raw/us_equities_tradingview_data_valuation.csv")

In [ ]:
@dataclass
class ValueRange:
    min: float
    max: float
    
    def validate(self, x):
        """
        Checks if inputs to variables that must lie within a specific range are valid
        
        :x: the value that must be checked as satisfying the specified range
        :raises ValueError: if the value does not lie within the specified range
        """
        if not (self.min <= x <= self.max):
            raise ValueError(f'{x} must be between 0 and 1 (including).')

In [ ]:
class QuantitativeAnalysis:
    def __init__(self, initial_capital: float=100000.00, capital_per_period: float=100.00, period: int=7, dividends_importance: bool=False, preferred_industries: list=["Technology Services, Electronic Technology"],
                volatility_tolerance: Annotated[float, ValueRange(0.0, 1.0)]=0.7, preferred_companies: list=["Apple, Google, Microsoft, Amazon"], diversification: Annotated[float, ValueRange(0.0, 1.0)]=0.4, investment_strategy: str="Growth"):
        """
        Includes several analysis functions that process select data across all data sets

        :initial_capital: the initial amount of cash to be invested by the client, in USD\n
        :capital_per_period: the amount of cash to be invested by the client at a fixed rate in addition to the initial capital invested, in USD\n
        :period: the frequency (in days) at which additional cash is invested, if desired\n
        :dividends_importance: specifies whether dividends are important to the client, dictating whether analysis algorithms should place greater importance on dividends\n
        :preferred_industries: specifies a list of industries that the analysis algorithms should prioritize when constructing the investment portfolio\n
        :volatility_tolerance: accepts a range of values from 0 to 1, with 1 implying maximum volatility tolerance (i.e. the client is willing to lose 100% of their investment to take on more risk)\n
        :preferred_companies: specifies a list of companies that the analysis algorithms will accomodate in the final portfolio irrespective of their score\n
        :diversification: accepts a range of values from 0 to 1, with 1 implying maximum diversification (i.e. funds will be distributed evenly across all industries and equally among all companies)\n
        :investment_strategy: specifies the investment strategy that will guide the output of the analysis algorithms, in which this analysis notebook strictly focuses on growth investing\n
        """
        
        self.initial_capital = initial_capital
        self.capital_per_period = capital_per_period
        self.period = period
        self.dividends_importance = dividends_importance
        self.preferred_industries = preferred_industries
        self.volatility_tolerance = volatility_tolerance
        self.preferred_companies = preferred_companies
        self.diversification = diversification
        self.preferred_companies = preferred_industries
        self.investment_strategy = investment_strategy
        
    def multiple_linear_regression(self):
        pass

    def fourier_transform(self):
        pass
    
    def rank(self, df: pd.DataFrame, col: str, normalize_only: bool=False, threshold: float=1.5,
             below_threshold: bool=True, filter_outliers: bool=True, normalize_after: bool=False,
             lower_quantile: float=0.05, upper_quantile: float=0.95) -> None:
        """
        The scoring algorithm for determining the weight of each equity in the construction of the portfolio for this specific column examined.
        Features a custom outlier-filtering algorithm that is robust to outliers in the data set while still returning normalized values.
        
        :df: The original dataframe\n
        :col: The name of the column being extracted from the dataframe provided\n
        :normalize_only: if True, does not apply a threshold to the screening algorithm, and only normalizes values with a minmax scaler\n
        :threshold: the minimum value that equities must have for that column in order to be considered for further analysis\n
        :below_threshold: if True, removes equities that are below the threshold for that column\n
        :filter_outliers: if True, does not consider equities in the data normalization algorithm, but assigns a min or max value to all outliers depending on the below_threshold parameter\n
        :normalize_after: if True, normalizes the data only after the threshold filter has been applied\n
        :lower_quantile: specifies the lower quantile of the distribution when filtering outliers\n
        :upper_quantile: specifies the upper quantile of the distribution when filtering outliers\n
        """
        
        #NOTE: should make an option for no threshold
        self.x = df[col]
        new_col = col + " Score"
        
        # normalization can be done either before or after equities have been filtered by the threshold
        # the difference is that by filtering initially, the min and max values of that smaller set will become 0 and 1 respectively
        df[new_col] = np.NaN # initialize the score column with only NaN values
        
        def outlier_filter(self):
            """
            Nested helper function to filter outliers
            """
            upper_fence = self.x.quantile(upper_quantile)
            lower_fence = self.x.quantile(lower_quantile)
            
            if below_threshold:
                df.loc[self.x > upper_fence, new_col] = 1 # outliers still need to be included in the data (max score assigned)
                df.loc[self.x < lower_fence, new_col] = 0 # lowest score assigned
            else:
                # inverse of the above
                df.loc[self.x > upper_fence, new_col] = 0
                df.loc[self.x < lower_fence, new_col] = 1

            # now only take the rows that are not outliers into the minmax scaler
            self.x = self.x[(self.x <= upper_fence) & (self.x >= lower_fence)]
            
            if normalize_only:
                normalize_after = False
                
            if normalize_after:
                if below_threshold:
                    # since we are only taking valid values, we consider the inverse of the values that are below the threshold to be valid values
                    self.x = self.x[self.x >= threshold]
                else:
                    self.x = self.x[self.x <= threshold]
        
        if filter_outliers:
            outlier_filter(self)
        
        self.y = np.array(self.x).reshape(-1, 1)
        self.y = preprocessing.MinMaxScaler().fit_transform(self.y)
 
        for col_idx, array_idx in zip(self.x.index, range(len(self.y))):
            df.at[col_idx, new_col] = self.y[array_idx]
        
        # if we are giving the minimum score to values below the threshold, assign 0 to those values
        if not normalize_only:
            if below_threshold:
                df.loc[df[col] <= threshold, new_col] = 0
            else:
                df.loc[df[col] >= threshold, new_col] = 0
    
    def time_series_forecast(self):
        pass

In [ ]:
class DataVisualization(QuantitativeAnalysis):
    def __init__(self):
        pass
    
    def score_distribution(self):
        pass

In [ ]:
class PortfolioConstruction(DataVisualization, QuantitativeAnalysis):
    def __init__(self):
        pass
    
    def asset_allocation(self):
        pass
    
    def construct_portfolio(self):
        pass

# Testing Zone

In [ ]:
test = QuantitativeAnalysis()
test.rank(overview_df, 'Price', normalize_only=True)
display(overview_df)
overview_df = overview_df.drop(columns=['Price Score'])

,Ticker,Description,Price,Change %,Change,Technical Rating,Volume,Volume*Price,Market Capitalization,Price to Earnings Ratio (TTM),Basic EPS (TTM),Number of Employees,Sector,Free Cash Flow (Annual YoY Growth),Free Cash Flow Margin (FY),Free Cash Flow (Quarterly YoY Growth),Price Score
0,AAPL,Apple Inc.,143.0000,10.059263,13.0700,Neutral,1377778266,1.970223e+11,2.264578e+12,23.914718,6.1445,164000.0,Electronic Technology,19.891773,28.261498,22.742534,1.000000
1,MSFT,Microsoft Corporation,242.7100,1.205070,2.8900,Neutral,639626996,1.552439e+11,1.806686e+12,27.591728,9.0309,221000.0,Technology Services,16.092876,32.858728,-43.134068,1.000000
2,GOOG,Alphabet Inc.,97.9500,10.391074,9.2200,Neutral,503871113,4.935418e+10,1.258830e+12,20.266028,5.0893,156500.0,Technology Services,56.412950,26.025291,-14.118590,0.704225
3,AMZN,"Amazon.com, Inc.",100.5500,19.702381,16.5500,Neutral,1457271131,1.465286e+11,1.025776e+12,93.884298,1.1147,1608000.0,Retail Trade,-156.804505,-3.134379,41.031417,0.723101
4,BRK.A,Berkshire Hathaway Inc.,465039.9800,-0.783208,-3670.9800,Buy,79667,3.704834e+10,6.758335e+11,NaN,-1194.0000,372000.0,Finance,-2.301857,9.469601,-10.582135,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8110,JZRO,Janus Henderson Net Zero Transition Resources ETF,25.3490,8.681112,2.0248,Buy,26175,6.635101e+05,NaN,NaN,NaN,NaN,Miscellaneous,NaN,NaN,NaN,0.177138
8111,XHYE,BondBloxx USD High Yield Bond Energy Sector ETF,37.7700,2.441009,0.9000,Strong Buy,17366,6.559138e+05,NaN,NaN,NaN,NaN,Miscellaneous,NaN,NaN,NaN,0.267315
8112,EWS,iShares Inc iShares MSCI Singapore ETF,20.2400,7.602339,1.4300,Sell,10576590,2.140702e+08,NaN,NaN,NaN,NaN,Miscellaneous,NaN,NaN,NaN,0.140046
8113,RESI,Kelly Residential & Apartment Real Estate ETF,10.9258,8.922519,0.8950,Buy,6014,6.570776e+04,NaN,NaN,NaN,NaN,Miscellaneous,NaN,NaN,NaN,0.072425


In [ ]:
overview_df = overview_df[:500]
summary_stats = overview_df.describe().T
summary_stats

,count,mean,std,min,25%,50%,75%,max
Price,500.0,1.109118e+03,2.079169e+04,3.710000e+00,6.058500e+01,1.084900e+02,2.022400e+02,4.650400e+05
Change %,500.0,6.346737e+00,1.001812e+01,-1.990489e+01,-2.787168e-01,4.911790e+00,1.114510e+01,7.203514e+01
Change,500.0,1.506680e+00,1.690525e+02,-3.670980e+03,-1.775000e-01,4.255000e+00,1.126500e+01,4.217000e+02
Volume,500.0,1.011209e+08,2.291622e+08,7.966700e+04,2.245486e+07,3.985203e+07,9.914405e+07,3.700686e+09
Volume*Price,500.0,9.486782e+09,3.189471e+10,4.051392e+07,2.591875e+09,4.389185e+09,8.681834e+09,6.167563e+11
Market Capitalization,500.0,7.455790e+10,1.647264e+11,1.346732e+10,1.958522e+10,3.383827e+10,6.266728e+10,2.264578e+12
Price to Earnings Ratio (TTM),436.0,4.266772e+01,1.332763e+02,1.176406e+00,1.582328e+01,2.348775e+01,3.616347e+01,2.325102e+03
Basic EPS (TTM),481.0,4.534197e+00,6.049386e+01,-1.194000e+03,1.736300e+00,4.025300e+00,8.372900e+00,4.801552e+02
Number of Employees,408.0,5.847286e+04,1.597499e+05,6.600000e+01,7.875000e+03,1.830000e+04,5.570000e+04,2.300000e+06
Free Cash Flow (Annual YoY Growth),449.0,1.041412e+02,7.521674e+02,-1.081875e+03,-2.361255e+01,7.359706e+00,4.568206e+01,1.331609e+04


In [ ]:
overview_df = overview_df.dropna()
test_df = overview_df.select_dtypes(exclude='object')
cols = test_df.columns
dict = {}

for column in cols[1:]:
    dict.update({column + " zscore": stats.zscore(test_df[column])})
    
stat_df = pd.DataFrame(dict)
stat_df

,Change % zscore,Change zscore,Volume zscore,Volume*Price zscore,Market Capitalization zscore,Price to Earnings Ratio (TTM) zscore,Basic EPS (TTM) zscore,Number of Employees zscore,Free Cash Flow (Annual YoY Growth) zscore,Free Cash Flow Margin (FY) zscore,Free Cash Flow (Quarterly YoY Growth) zscore
0,0.538499,0.074397,5.084960,4.896091,11.584310,-0.154363,-0.111517,0.595468,-0.105311,0.826602,0.038341
1,-0.461151,-0.149478,2.171102,3.801375,9.158020,-0.129570,-0.006802,0.929604,-0.110236,1.118100,-0.210158
2,0.575961,-0.010271,1.635204,1.026756,6.255035,-0.178966,-0.149799,0.551503,-0.057967,0.684811,-0.100706
3,1.627220,0.150928,5.398758,3.573010,5.020125,0.317438,-0.293993,9.060240,-0.334371,-1.164124,0.107330
5,3.387979,0.743165,14.254656,15.894347,2.373338,0.055019,-0.203764,0.216136,-0.022455,-0.378349,0.516098
...,...,...,...,...,...,...,...,...,...,...,...
494,-0.190064,-0.146180,-0.264942,-0.214935,-0.342504,-0.126723,-0.219672,-0.314905,-0.030153,0.608858,-0.050458
495,1.139896,0.348414,-0.305469,-0.205005,-0.343078,0.584240,-0.281230,-0.353313,0.281769,-0.417456,2.090446
496,-0.063832,0.131136,-0.334294,-0.221479,-0.343280,-0.137430,0.147522,-0.354492,-0.094160,1.220413,-0.080041
497,-0.680319,-0.251520,-0.335983,-0.238511,-0.343809,-0.132336,-0.011231,-0.322931,-0.142825,0.164818,-0.044755


In [ ]:
def distplot(df, data_name):
    df = df.select_dtypes(exclude='object')[:500]
    n = len(df)
    
    for column in df.columns:
        test.rank(df, col=column, normalize_only=True, upper_quantile=0.99, lower_quantile=0.01)
        
    score_data_length = len(df.axes[1])
    input_df = df.T[int(score_data_length/2 + 1):].T
    hist_data = [input_df[x] for x in input_df.columns]
    
    group_labels = [x for x in input_df.columns]
    colors = ['#333F44', '#37AA9C', '#94F3E4']

    fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)
    fig.update_layout(title_text=f'Distribution for Normalized {data_name} of {n} Companies in the S&P500', template='plotly_dark')
    fig.show()

dfs = [
    overview_df,
    balance_sheet_df,
    dividends_df,
    income_statement_df,
    margins_df,
    performance_df,
    valuation_df
    ]

dfs_names = [
    "Overview Data",
    "Balance Sheet Data",
    "Dividends Data",
    "Income Statement Data",
    "Margins Data",
    "Performance Data",
    "Valuation Data"
    ]

for df, name in zip(dfs, dfs_names):
    df.dropna(inplace=True)
    distplot(df, name)

These plots reveal a number of things. First, they indicate that there are cases where most companies perform either reasonably well or generally bad for certian financial ratios, as represented by the width of the distribution plot for each ratio. The smaller it is, the more likely that most companies perform similarly. Secondly, most graphs have cases where outliers exist for each category, in which these companies may perform exceptionally well compared to others for a certain financial ratio--being a potentially strong pick.

The next step is to investigate if a correlation exists between cases where most companies score low for a certain ratio, and outliers of that segment performing exceptionally well.

In [ ]:
def heatmap_plot(df, data_name, number_of_companies):
    df = df[:number_of_companies]
    z = []
    tickers = df['Ticker']
    df.index = df['Ticker']
    df = df.select_dtypes(exclude='object')
    for column in df.columns:
        test.rank(df, col=column, normalize_only=True, upper_quantile=0.95, lower_quantile=0.01)

    score_data_length = len(df.axes[1])
    input_df = df.T[int(score_data_length/2 + 1):].T
    for column in input_df.columns:
        z.append(input_df[column].round(3))
    
    fig = px.imshow(z, text_auto=True, template='plotly_dark', title=f'Heat Map of Normalized {data_name} for the Top {number_of_companies} Companies by Market Capitalization in the S&P500 Index',
                x=[x for x in tickers],
                y=[x for x in df.columns[int(score_data_length/2 + 1):]]
               )
    fig.show()


mega_df = pd.concat(dfs, axis=1)
mega_df = mega_df.loc[:,~mega_df.columns.duplicated()].copy()
#len(mega_df.columns)

#Index(['Ticker', 'Description', 'Price', 'Basic EPS (TTM)', 'Change %',
#       'Market Capitalization', 'Price to Earnings Ratio (TTM)',
#       'EPS Diluted (FY)'],
#      dtype='object')
mega_df = mega_df.dropna()
heatmap_plot(mega_df, f'Complete Equity Data ({len(mega_df.columns)} Data Points)', 50)

for df, names in zip(dfs, dfs_names):
    heatmap_plot(df, names, 50)

These heat plots reveal that for certain categories of data, particularly valuation, income statement and balance sheet data, the top companies by market capitalization tend to have the highest scores in those categories. Although this may indicate that such companies with the highest aggregated normalized scores are the best pick for an investment portfolio, this assumption must be validated against their past 3-month performance to see if these scores did indeed dictate a positive change in the price of an equity--indicating a positive return on investment. A multiple linear regression can be used, but first, a 3D plot can be used to closely analyze the correlation of two of the most important pieces of equity data when picking stocks with the corresponding change in the price of such stocks.

In [ ]:
def scatter_3d(df, x, y, z):
    df.index = df['Ticker']
    df = df.select_dtypes(exclude='object')
    for column in df.columns:
        test.rank(df, col=column, normalize_only=True, upper_quantile=0.95, lower_quantile=0.01)
    fig = px.scatter_3d(df, x=x, y=y, z=z,
                title='3D Scatter Plot of Normalized Equity Data',
                template='plotly_dark',
                size_max=18,
                color='3-Month Performance Score',
                opacity=0.7)

    #color='petal_length', size='petal_length', 
    fig.show()

    #fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

#overview_df = overview_df.drop(columns='Ticker')
#overview_df.columns
scatter_3d(mega_df, 'Price to Earnings Ratio (TTM) Score', 'Free Cash Flow Margin (FY) Score', '3-Month Performance Score')

There appears to be a reasonably strong correlation between the normalized Free Cash Flow Margin (FY) score and the corresponding 3-Month Performance score, indicating that a high Free Cash FLow Margin (FY) Score may be a good metric to consider when picking a stock. The same applies for the correlation between the Price to Earnings Ratio (TTM) score and the 3-Month Performance score. Plotting a regression line can be used to validate this hypothesis.